# Allen Ben Philipose - 18BIS0043
#### Lab FAT, L53+L54
#### ECE3502 - IoT Domain Analyst
#### Submitted to: Prof. Pradheep T

# Aim

In the given dataset, use random forest and predict the number of cases for 
- 01.03.2021,Belgium (Reg.No: 18BIS0043)

# Algorithm Explanation

Random forests, or random decision forests, are an ensemble learning technique for classification, regression, and other tasks. It works by training a large number of decision trees and then outputing the class that is the mode of the classes (classification) or the mean/average prediction (regression) of the individual trees. Random decision forests compensate for decision trees' inclination for excessive fit to their training set. Although random forests surpass decision trees in practice, their accuracy is lower than that of gradient boosted trees. However, the features of the data might have an effect on their performance.

Each tree in the random forest generates a class prediction, and the class with the most votes becomes the prediction of our model. A large number of substantially uncorrelated models (trees) acting in conjunction outperforms any of the component models individually. A random forest is a meta estimator that fits a number of decision tree classifiers to different subsamples of the dataset and then utilises averaging to increase predicted accuracy and avoid overfitting.

"Random Forest is a classifier that combines many decision trees on diverse subsets of a given dataset and uses the average to increase the dataset's predicted accuracy." Rather of depending on a single decision tree, the random forest collects the forecasts from each tree and forecasts the final output based on the majority vote of forecasts. 

<br />

*Info from documentations*

# Code and Result

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, f1_score 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV, KFold 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

In [61]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [69]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr

## Dataset

In [21]:
allen = pd.read_csv("Dataset.csv")

In [22]:
allen.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,14-12-2020,14,12,2020,746,6,Afghanistan,AF,AFG,38041757.0,Asia,9.013779
1,13-12-2020,13,12,2020,298,9,Afghanistan,AF,AFG,38041757.0,Asia,7.052776
2,12-12-2020,12,12,2020,113,11,Afghanistan,AF,AFG,38041757.0,Asia,6.868768
3,11-12-2020,11,12,2020,63,10,Afghanistan,AF,AFG,38041757.0,Asia,7.134266
4,10-12-2020,10,12,2020,202,16,Afghanistan,AF,AFG,38041757.0,Asia,6.968658


In [23]:
# Dropping all rows except Europe

allen1 = allen[(allen['continentExp'] != 'Europe')].index
allen.drop(allen1, inplace=True)

In [25]:
# Dropping all columns except date, cases and country

allen = allen.drop(['deaths','geoId','countryterritoryCode','popData2019','continentExp','Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'],axis=1)

In [26]:
allen.head()

,dateRep,day,month,year,cases,countriesAndTerritories
340,14-12-2020,14,12,2020,788,Albania
341,13-12-2020,13,12,2020,879,Albania
342,12-12-2020,12,12,2020,802,Albania
343,11-12-2020,11,12,2020,873,Albania
344,10-12-2020,10,12,2020,752,Albania


## Label Encoding

In [27]:
l1 = preprocessing.LabelEncoder()
f1 = l1.fit_transform(allen['countriesAndTerritories']) 
f1 = pd.DataFrame(data=f1, columns=['countriesAndTerritories'])
allen['countriesAndTerritories'] = f1['countriesAndTerritories']

In [35]:
print(list(l1.inverse_transform(range(0,55))))

['Albania', 'Andorra', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium', 'Bosnia_and_Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Faroe_Islands', 'Finland', 'France', 'Georgia', 'Germany', 'Gibraltar', 'Greece', 'Guernsey', 'Holy_See', 'Hungary', 'Iceland', 'Ireland', 'Isle_of_Man', 'Italy', 'Jersey', 'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'North_Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San_Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine', 'United_Kingdom']


In [51]:
# Assigned Country

print(list(l1.inverse_transform([6])))

['Belgium']


In [36]:
allen.head()

,dateRep,day,month,year,cases,countriesAndTerritories
340,14-12-2020,14,12,2020,788,1.0
341,13-12-2020,13,12,2020,879,1.0
342,12-12-2020,12,12,2020,802,1.0
343,11-12-2020,11,12,2020,873,1.0
344,10-12-2020,10,12,2020,752,1.0


## Pre-processing

In [43]:
allen.dropna(subset = ["cases"], inplace=True)
allen.dropna(subset = ["day"], inplace=True)
allen.dropna(subset = ["month"], inplace=True)
allen.dropna(subset = ["year"], inplace=True)
allen.dropna(subset = ["countriesAndTerritories"], inplace=True)

In [44]:
allen.head()

,dateRep,day,month,year,cases,countriesAndTerritories
340,14-12-2020,14,12,2020,788,1.0
341,13-12-2020,13,12,2020,879,1.0
342,12-12-2020,12,12,2020,802,1.0
343,11-12-2020,11,12,2020,873,1.0
344,10-12-2020,10,12,2020,752,1.0


## Train-test-split

In [45]:
x = allen.drop(['dateRep','cases'],axis=1)

In [46]:
y = allen['cases']

In [47]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2,random_state=21)

## Model

In [72]:
model0 = RandomForestClassifier()
model0.fit(xtrain,ytrain)
p0 = model0.predict(xtest)

## Prediction (18BIS0043)

In [73]:
allen2 = pd.DataFrame(columns=['day','month','year','countriesAndTerritories','cases'])

In [74]:
values = {"day": 1, 
          "month": 3,
          "year": 2021,
          "countriesAndTerritories": 6,
          "cases": 34,
}
allen2 = allen2.append(values, ignore_index=True)

In [75]:
pred = model0.predict(allen2.drop(['cases'],axis=1))
allen2['cases']=pred

In [76]:
allen2.head()

,day,month,year,countriesAndTerritories,cases
0,1,3,2021,6,1


# Inferences

Using the given dataset, we have successfully classified the users based on their shopping patterns to predict the Online Shopping websites they use. The classification model is based on the ML Algorithm - **Decision Tree** with accuracies upto 36%. Availability of training data, and its quality is the primary reason that determines the accuracy. Lower dataset quality indirectly implies a lower accuracy for the model.

We have also taken the example of a female user, 34 years old with a spending amount of Rs. 4900, and predicted the online shopping website as "NYKAA" based on the learning from the dataset.

In [82]:
m = mean_squared_error(ytest,model0.predict(xtest))
print("MSE: %.5f"%m)

MSE: 703709.70828


In [80]:
print(classification_report(ytest.values.ravel(),p0,target_names=range(0,721)))

TypeError: object of type 'int' has no len()

# Viva

Which ML algorithm is the most effective for this application given the same dataset. Justify your choice of ML algorithm.

#### Random Forest

In [97]:
model1 = RandomForestClassifier()
model1.fit(xtrain,ytrain)
p1 = model1.predict(xtest)

In [98]:
print(confusion_matrix(ytest,p1))

[[103   1   2 ...   0   0   0]
 [  8   3   0 ...   0   0   0]
 [  3   5   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   1   0]]


In [100]:
print(classification_report(ytest.values.ravel(),p1,target_names=list(str(int) for int in range(0,722))))

   0.00         0
         355       0.00      0.00      0.00         0
         356       0.00      0.00      0.00         1
         357       0.00      0.00      0.00         0
         358       0.00      0.00      0.00         0
         359       0.00      0.00      0.00         0
         360       0.00      0.00      0.00         2
         361       0.00      0.00      0.00         1
         362       0.00      0.00      0.00         1
         363       0.00      0.00      0.00         0
         364       0.00      0.00      0.00         1
         365       0.00      0.00      0.00         0
         366       0.00      0.00      0.00         0
         367       0.00      0.00      0.00         1
         368       0.00      0.00      0.00         0
         369       0.00      0.00      0.00         1
         370       0.00      0.00      0.00         1
         371       0.00      0.00      0.00         1
         372       0.00      0.00      0.00         1
         3

#### Decision Tree

In [372]:
model2 = DecisionTreeClassifier()
model2.fit(xtrain,ytrain)
p2 = model2.predict(xtest)

In [373]:
print(confusion_matrix(ytest,p2))

[[1 1 0 0 1 2 0]
 [0 1 0 2 0 0 1]
 [0 1 2 0 0 1 1]
 [1 0 2 5 0 0 0]
 [1 1 1 1 3 0 0]
 [3 1 0 1 0 2 1]
 [0 1 0 0 0 0 1]]


In [374]:
print(classification_report(ytest.values.ravel(),p2,target_names=['0','1','2','3','4','5','6']))

              precision    recall  f1-score   support

           0       0.17      0.20      0.18         5
           1       0.17      0.25      0.20         4
           2       0.40      0.40      0.40         5
           3       0.56      0.62      0.59         8
           4       0.75      0.43      0.55         7
           5       0.40      0.25      0.31         8
           6       0.25      0.50      0.33         2

    accuracy                           0.38        39
   macro avg       0.38      0.38      0.37        39
weighted avg       0.43      0.38      0.39        39



#### Naive Bayes

In [375]:
model3 = GaussianNB()
model3.fit(xtrain,ytrain)
p3 = model3.predict(xtest)

In [376]:
print(confusion_matrix(ytest,p3))

[[0 3 0 0 2 0 0]
 [1 1 0 2 0 0 0]
 [0 1 4 0 0 0 0]
 [1 2 1 4 0 0 0]
 [1 3 0 0 1 0 2]
 [3 1 0 1 1 2 0]
 [0 1 1 0 0 0 0]]


In [377]:
print(classification_report(ytest.values.ravel(),p3,target_names=['0','1','2','3','4','5','6']))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.08      0.25      0.12         4
           2       0.67      0.80      0.73         5
           3       0.57      0.50      0.53         8
           4       0.25      0.14      0.18         7
           5       1.00      0.25      0.40         8
           6       0.00      0.00      0.00         2

    accuracy                           0.31        39
   macro avg       0.37      0.28      0.28        39
weighted avg       0.46      0.31      0.33        39



#### K-Nearest Neighbour

In [378]:
model4 = KNeighborsClassifier()
model4.fit(xtrain,ytrain)
p4 = model4.predict(xtest)

In [379]:
print(confusion_matrix(ytest,p4))

[[0 2 0 1 2 0 0]
 [1 1 1 1 0 0 0]
 [0 2 1 1 0 0 1]
 [3 2 1 1 0 0 1]
 [1 2 2 0 1 0 1]
 [4 0 1 0 1 1 1]
 [1 0 0 1 0 0 0]]


In [380]:
print(classification_report(ytest.values.ravel(),p4,target_names=['0','1','2','3','4','5','6']))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.11      0.25      0.15         4
           2       0.17      0.20      0.18         5
           3       0.20      0.12      0.15         8
           4       0.25      0.14      0.18         7
           5       1.00      0.12      0.22         8
           6       0.00      0.00      0.00         2

    accuracy                           0.13        39
   macro avg       0.25      0.12      0.13        39
weighted avg       0.32      0.13      0.15        39



#### Logistic Regression

In [381]:
model5 = LogisticRegression()
model5.fit(xtrain,ytrain)
p5 = model5.predict(xtest)

In [382]:
print(confusion_matrix(ytest,p5))

[[1 0 0 1 0 0 3]
 [0 0 0 3 0 0 1]
 [0 0 0 5 0 0 0]
 [2 0 0 5 0 0 1]
 [2 0 0 1 0 0 4]
 [2 0 0 4 0 0 2]
 [1 0 0 1 0 0 0]]


In [383]:
print(classification_report(ytest.values.ravel(),p5,target_names=['0','1','2','3','4','5','6']))

              precision    recall  f1-score   support

           0       0.12      0.20      0.15         5
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         5
           3       0.25      0.62      0.36         8
           4       0.00      0.00      0.00         7
           5       0.00      0.00      0.00         8
           6       0.00      0.00      0.00         2

    accuracy                           0.15        39
   macro avg       0.05      0.12      0.07        39
weighted avg       0.07      0.15      0.09        39



## K-Fold Analysis

In [89]:
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=21)

#### Random Forest

In [94]:
acc = np.mean(cross_val_score(model1, xtrain, ytrain, scoring='accuracy', cv=kfold))

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [95]:
allen11 = pd.DataFrame({'Model':'Random Forest',
                      'Accuracy':[acc]})

In [92]:
allen11.head()

,Model,Accuracy
0,Random Forest,0.162065


#### Decision Tree

In [387]:
acc = np.mean(cross_val_score(model2, xtrain, ytrain, scoring='accuracy', cv=kfold))

In [388]:
allen12 = pd.DataFrame({'Model':'Decision Tree',
                      'Accuracy':[acc]})

#### Naive Bayes

In [389]:
acc = np.mean(cross_val_score(model3, xtrain, ytrain, scoring='accuracy', cv=kfold))

In [390]:
allen13 = pd.DataFrame({'Model':'Naive Bayes',
                      'Accuracy':[acc]})

#### K-Nearest Neighbour

In [391]:
acc = np.mean(cross_val_score(model4, xtrain, ytrain, scoring='accuracy', cv=kfold))

In [392]:
allen14 = pd.DataFrame({'Model':'K-Nearest Neighbour',
                      'Accuracy':[acc]})

#### Logistic Regression

In [393]:
acc = np.mean(cross_val_score(model5, xtrain, ytrain, scoring='accuracy', cv=kfold))

In [394]:
allen15 = pd.DataFrame({'Model':'Logistic Regression',
                      'Accuracy':[acc]})

#### Concat

In [395]:
al = pd.concat([allen11,allen12,allen13,allen14,allen15],axis=0).reset_index()
al = al.drop('index',axis=1)
al

,Model,Accuracy
0,Random Forest,0.277425
1,Decision Tree,0.288970
2,Naive Bayes,0.274501
3,K-Nearest Neighbour,0.199486
4,Logistic Regression,0.184866


Using the K-Fold Analysis, a method for comparison, we can compare the algorithms based on the accuracies offered by them. Here, the highest accuracy offered is by Decision Tree ML algorithm, closely followed by Random Forest ML algorithm,for the given scenario and dataset.